In [10]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
data = pd.read_csv("/Users/yashp/OneDrive/Desktop/cocktail.csv")

# Extract the relevant columns from the dataset
ingredients = data["ingredients"].tolist()
instructions = data["instructions"].tolist()

# Tokenize the ingredients and instructions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ingredients + instructions)
vocab_size = len(tokenizer.word_index) + 1
ingredients_seq = tokenizer.texts_to_sequences(ingredients)
instructions_seq = tokenizer.texts_to_sequences(instructions)

# Pad the sequences so that they are all the same length
max_length = max([len(seq) for seq in instructions_seq])
ingredients_seq = pad_sequences(ingredients_seq, padding="post", maxlen=max_length)
instructions_seq = pad_sequences(instructions_seq, padding="post", maxlen=max_length)

# Define the neural network architecture
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_length))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(vocab_size, activation="softmax"))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

# Train the model
history = model.fit(ingredients_seq, instructions_seq, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Generate new cocktail recipes
input_ingredients = ["vodka", "cranberry juice", "lime juice"]
input_seq = tokenizer.texts_to_sequences([input_ingredients])[0]
input_seq = pad_sequences([input_seq], padding="post", maxlen=max_length)
output_seq = model.predict_classes(input_seq)[0]
output_recipe = tokenizer.sequences_to_texts([output_seq])[0]

print(f"Input ingredients: {', '.join(input_ingredients)}")
print(f"Generated recipe: {output_recipe}")

ValueError: A target array with shape (5, 26) was passed for an output of shape (None, 39) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [11]:
import os
import cv2
import pixellib
from pixellib.instance import instance_segmentation

input_folder = "/Users/yashp/OneDrive/Desktop/RESEARCH/combined/train/good/"
output_folder = "/Users/yashp/OneDrive/Desktop/RESEARCH/Segmentation/"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the pre-trained Mask R-CNN model
seg = instance_segmentation()
seg.load_model("mask_rcnn_coco.h5")

# Iterate over the images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Perform instance segmentation
        result = seg.segmentImage(image)

        # Get the main object mask
        main_object_mask = result[0]['masks'][:, :, 0]

        # Apply the mask to the original image
        output_image = cv2.bitwise_and(image, image, mask=main_object_mask)

        # Save the output image in the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, output_image)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'mask_rcnn_coco.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [12]:
import cv2
import os

# Input and output directories
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/combined/train/good/'
output_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/Segmentation/'

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through all contours
    for i, contour in enumerate(contours):
        # Get the bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the contour and save it as a new image
        crop_img = img[y:y+h, x:x+w]
        cv2.imwrite(output_dir + filename.split('.')[0] + '_' + str(i) + '.jpg', crop_img)


In [13]:
import cv2
import os

# Input and output directories
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/combined/train/good/'
output_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/Segmentation/'

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black mask with the same size as the original image
    mask = np.zeros(img.shape[:2], dtype=np.uint8)

    # Loop through all contours
    for i, contour in enumerate(contours):
        # Draw the contour on the mask
        cv2.drawContours(mask, [contour], 0, 255, -1)

        # Copy the masked region from the original image to a new image with the same size as the original
        masked_img = cv2.bitwise_and(img, img, mask=mask)
        x, y, w, h = cv2.boundingRect(contour)
        cropped_img = masked_img[y:y+h, x:x+w]
        new_img = np.zeros_like(img)
        new_img[y:y+h, x:x+w] = cropped_img

        # Save the new image to the output directory
        cv2.imwrite(output_dir + filename.split('.')[0] + '_' + str(i) + '.jpg', new_img)

        # Reset the mask
        mask = np.zeros(img.shape[:2], dtype=np.uint8)


In [14]:
import cv2
import os

# Input and output directories
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/combined/train/good/'
output_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/newsegment'

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original image
    cv2.drawContours(img, contours, -1, (0, 0, 255), 2)

    # Save the original image to the output directory
    cv2.imwrite(output_dir + filename, img)


In [15]:
import cv2
import os

# Input and output directories
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/combined/train/good/'
output_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/segment2/'

# Create a background subtractor object
backSub = cv2.createBackgroundSubtractorMOG2()

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Apply background subtraction to get the foreground mask
    fgMask = backSub.apply(img)

    # Threshold the foreground mask to create a binary mask
    thresh = cv2.threshold(fgMask, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

    # Find contours in the binary mask
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area (i.e., the main object)
    max_area = 0
    max_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_contour = contour

    # Create a black mask with the same size as the original image
    mask = np.zeros(img.shape[:2], dtype=np.uint8)

    # Draw the main contour on the mask
    cv2.drawContours(mask, [max_contour], 0, 255, -1)

    # Copy the masked region from the original image to a new image with the same size as the original
    masked_img = cv2.bitwise_and(img, img, mask=mask)
    new_img = np.zeros_like(img)
    x, y, w, h = cv2.boundingRect(max_contour)
    new_img[y:y+h, x:x+w] = masked_img[y:y+h, x:x+w]

    # Save the new image to the output directory
    cv2.imwrite(output_dir + filename, new_img)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\drawing.cpp:2605: error: (-215:Assertion failed) reader.ptr != NULL in function 'cvDrawContours'


In [19]:
import cv2
import os

# Input and output directories
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/combined/test/anomaly/'
output_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/segment2/test/anomaly/'

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Find contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours were found
    if len(contours) > 0:
        # Draw the contours on the original image
        cv2.drawContours(img, contours, -1, (0, 0, 255), 2)

    # Save the original image to the output directory
    cv2.imwrite(output_dir + filename, img)


In [20]:
import cv2
import os

# Input directory
input_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/segment2/train/good/'

# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(input_dir + filename)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Set the thresholded pixels to black
    img[thresh == 255] = 0

    # Save the image to the same directory with the prefix 'black_'
    cv2.imwrite(input_dir + 'black_' + filename, img)
